### Importing libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn as sns
import scipy
from scipy.stats import pearsonr
import sklearn
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as n
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV

import joblib
import os

### Analysis of the dataset

In [2]:
def read_data(file_path):
    return pd.read_excel(file_path)

def display_data(data):
    return data.head()

def drop_columns(data, columns, axis):
    return data.drop(columns=columns, axis=axis)

### Check distribution of data

In [3]:
def check_distribution(data):
    fig, axes = plt.subplots(3, 5, figsize=(20,10), sharey=True)
    axes = axes.ravel()
    for i, col in enumerate(data.columns[0:13]):
        if col == 'RelapseFreeSurvival (outcome)' or col == 'Age':
            sns.histplot(data=data, x=col, ax=axes[i])
        else:
            sns.countplot(data=data, x=col, ax=axes[i])
        axes[i].set_title(f'Count Plot for {col}')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Count')

    plt.tight_layout()
    return fig

### Handling missing values

In [4]:
def replace_nan_values(data):
    """Replace 999 values with NaN and return the modified DataFrame."""
    data_copy = data.copy()
    data_copy.replace(999, np.nan, inplace=True)
    return data_copy

def get_description_data(data):
    return data.iloc[:, 0:13].info()

def identify_categorical_continuous_features(data):
    """Identify categorical and continuous features based on data type and unique values."""
    # First, separate numeric and non-numeric columns
    numeric_columns = data.select_dtypes(include=['int64', 'float64']).columns
    non_numeric_columns = data.select_dtypes(exclude=['int64', 'float64']).columns
    
    # Among numeric columns, identify categorical (few unique values) and continuous
    categorical_features = [
        col for col in numeric_columns
        if data[col].nunique() < 20
    ] + list(non_numeric_columns)  # Add all non-numeric columns to categorical
    
    continuous_features = [
        col for col in numeric_columns
        if col not in categorical_features
    ]
    
    return categorical_features, continuous_features

def check_missing_data(data):
    null_counts = data.isnull().sum()
    null_counts = null_counts[null_counts > 0]
    null_percentage = null_counts / len(data)

    return null_counts, null_percentage

def init_imputers():
    mice_imputer = IterativeImputer(max_iter=50, random_state=0)
    mean_imputer = SimpleImputer(strategy='mean')
    return mice_imputer, mean_imputer

def impute_features_and_target(data, target_column, categorical_features, continuous_features):
    """Handle missing values for both features and target."""
    # Separate features and target
    X = data.drop(target_column, axis=1)
    y = data[target_column]
    
    # Impute features
    X_imputed = impute_features(X, categorical_features, continuous_features)
    
    # Impute target
    y_imputed = impute_target(y)
    
    # Combine imputed features and target
    final_data = X_imputed.copy()
    final_data[target_column] = y_imputed
    
    return final_data

def impute_features(X, categorical_features, continuous_features):
    """Impute missing values in features using appropriate strategies."""
    # Set up column transformer
    column_transformer = set_column_transformer(categorical_features, continuous_features)
    
    # Apply imputation
    X_imputed = apply_imputation(X, column_transformer)
    
    # Convert back to DataFrame
    return convert_to_dataframe(X_imputed, categorical_features, continuous_features)

def impute_target(y):
    """Impute missing values in target using mode strategy."""
    target_imputer = SimpleImputer(strategy='most_frequent')
    y_imputed = target_imputer.fit_transform(y.values.reshape(-1, 1))
    return y_imputed.ravel().astype(int)

def set_column_transformer(categorical_features, continuous_features):
    """Set up column transformer with appropriate imputation strategies."""
    # Initialize imputers
    categorical_imputer = SimpleImputer(strategy='most_frequent')  # Use mode for categorical
    continuous_imputer = SimpleImputer(strategy='mean')  # Use mean for continuous
    
    # Create transformers list
    transformers = []
    
    # Add categorical imputer if there are categorical features
    if categorical_features:
        transformers.append(('cat', categorical_imputer, categorical_features))
    
    # Add continuous imputer if there are continuous features
    if continuous_features:
        transformers.append(('cont', continuous_imputer, continuous_features))
    
    # Create and return column transformer
    return ColumnTransformer(
        transformers=transformers,
        remainder='passthrough'
    )

def apply_imputation(data, column_transformer):
    """Apply the column transformer to impute missing values."""
    return column_transformer.fit_transform(data)

def convert_to_dataframe(data, categorical_features, continuous_features):
    """Convert imputed array back to DataFrame with proper column names."""
    # Combine all feature names in the correct order
    all_features = categorical_features + continuous_features
    
    # Create DataFrame with original column names
    return pd.DataFrame(data, columns=all_features)

### Handling outliers

In [5]:
def visualise_outliers(data):
    # Example Z-score threshold for detecting extreme outliers
    z_threshold = 3

    # Calculate the number and percentage of extreme outliers for each column
    extreme_outliers_info = {}

    for col in data.select_dtypes(include=['int64', 'float64']).columns[1:]:
        z_scores = np.abs((data[col] - data[col].mean()) / data[col].std())
        outliers_count = (z_scores > z_threshold).sum()
        total_count = len(data[col].dropna())
        outliers_percentage = (outliers_count / total_count) * 100
        extreme_outliers_info[col] = {
            'count': outliers_count,
            'max_z_score': z_scores.max(),  # Track the maximum Z-score for the most extreme outlier
            'percentage': outliers_percentage,
            'outlier_indices': data[col].index[z_scores > z_threshold]  # Indices of outliers
        }

    # Sort columns by the maximum Z-score to ensure we get the most extreme outliers
    sorted_columns = sorted(
        extreme_outliers_info, 
        key=lambda x: (extreme_outliers_info[x]['count'], extreme_outliers_info[x]['max_z_score']), 
        reverse=True
    )

    # Choose the top N columns to visualize (e.g., top 24)
    top_columns = sorted_columns[:24]

    # Adjust the number of rows/columns for up to 24 plots (e.g., 6 rows, 4 columns)
    fig, axes = plt.subplots(nrows=6, ncols=4, figsize=(20, 30))
    fig.subplots_adjust(hspace=0.4, wspace=0.4)

    axes = axes.ravel()

    # Plot boxplots for the top columns with the most extreme outliers
    for i, col in enumerate(top_columns):
        outliers_count = extreme_outliers_info[col]['count']
        outliers_percentage = extreme_outliers_info[col]['percentage']
        
        # Create the boxplot
        sns.boxplot(y=col, x='pCR (outcome)', data=data, ax=axes[i], showfliers=False)  # Hide default fliers
        
        # Plot outliers in red
        outlier_indices = extreme_outliers_info[col]['outlier_indices']
        outliers = data.loc[outlier_indices]
        sns.scatterplot(
            y=outliers[col], 
            x=outliers['pCR (outcome)'], 
            ax=axes[i], 
            color='red', 
            edgecolor='w', 
            s=50, 
            alpha=0.8, 
            label='Outliers'
        )
        
        # Add title with outlier information
        axes[i].set_title(
            f"Column: {col}\nOutliers: {outliers_count} ({outliers_percentage:.2f}%)"
        )

    # Hide any unused subplots if you have fewer top columns than subplots
    for j in range(len(top_columns), len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    return fig

def count_outliers_using_iqr_range(data, iqr_multiple=1.5):
    outliers_dict = {}
    
    for column in data.select_dtypes(include=['int64', 'float64']).columns:
        try:
            Q1 = data[column].quantile(0.25)
            Q3 = data[column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - iqr_multiple * IQR
            upper_bound = Q3 + iqr_multiple * IQR
            iqr_outliers = data[column][(data[column] < lower_bound) | (data[column] > upper_bound)]
            outliers_dict[column] = len(iqr_outliers)
        except:
            outliers_dict[column] = 0
    
    return outliers_dict

def count_outliers_using_z_score(data, z_threshold=3):
    outliers_dict = {}
    
    for column in data.select_dtypes(include=['int64', 'float64']).columns:
        try:
            column_data = data[column].dropna()
            z_scores = np.abs((column_data - column_data.mean()) / column_data.std())
            z_score_outliers = column_data[z_scores > z_threshold]
            outliers_dict[column] = len(z_score_outliers)
        except:
            outliers_dict[column] = 0
    
    return outliers_dict

def handle_z_score_outliers(data, method='clip', z_threshold=3):
    df_processed = data.copy()
    
    for column in data.select_dtypes(include=['int64', 'float64']).columns:
        try:
            column_data = data[column].dropna()
            z_scores = np.abs((column_data - column_data.mean()) / column_data.std())
            extreme_outlier_mask = z_scores > z_threshold
            
            if method == 'clip':
                # Clip extreme values to z-score threshold boundaries
                mean = column_data.mean()
                std = column_data.std()
                df_processed[column] = df_processed[column].apply(
                    lambda x: min(max(x, mean - z_threshold*std), mean + z_threshold*std)
                )
            
            elif method == 'remove':
                # Remove rows with extreme outliers
                df_processed = df_processed[~extreme_outlier_mask]
            
            elif method == 'winsorize':
                # Winsorize extreme values
                df_processed[column] = stats.mstats.winsorize(
                    df_processed[column], 
                    limits=[0.05, 0.05]
                )
        
        except Exception as e:
            print(f"Error processing column {column}: {e}")
    
    return df_processed

### Data augmentation

In [6]:
def augment_data(X, y, minority_class=1, majority_class=0, strategy='smote'):
    """
    Augment the dataset to handle class imbalance.
    
    Parameters:
    -----------
    X : pandas.DataFrame
        Feature matrix
    y : pandas.Series
        Target variable
    minority_class : int, default=1
        Label of the minority class
    majority_class : int, default=0
        Label of the majority class
    strategy : str, default='smote'
        Strategy to use for augmentation ('smote' or 'upsample')
    
    Returns:
    --------
    tuple : (X_balanced, y_balanced)
        Balanced dataset
    """
    if strategy == 'smote':
        from imblearn.over_sampling import SMOTE
        smote = SMOTE(random_state=42)
        X_balanced, y_balanced = smote.fit_resample(X, y)
        
    elif strategy == 'upsample':
        # Combine features and target for resampling
        data = pd.concat([X, pd.Series(y, name='target')], axis=1)
        
        # Separate majority and minority classes
        majority = data[data.target == majority_class]
        minority = data[data.target == minority_class]
        
        # Upsample minority class
        minority_upsampled = resample(
            minority,
            replace=True,
            n_samples=len(majority),
            random_state=42
        )
        
        # Combine majority and upsampled minority
        data_balanced = pd.concat([majority, minority_upsampled])
        
        # Separate features and target
        X_balanced = data_balanced.drop('target', axis=1)
        y_balanced = data_balanced.target
        
    else:
        raise ValueError(f"Unknown strategy: {strategy}")
    
    return X_balanced, y_balanced

### Scaling and normalisation

In [7]:
def normalise_data(data, continuous_features):
    """Normalize only the continuous features while preserving categorical ones."""
    # Create a copy of the data
    df_normalized = data.copy()
    
    # Only normalize continuous features
    if continuous_features:
        scaler = StandardScaler()
        df_normalized[continuous_features] = scaler.fit_transform(data[continuous_features])
    
    return df_normalized

### Feature selection and dimensionality reduction

In [8]:
def create_heatmap(data):
    matrix = data.iloc[:, 0:12].corr(numeric_only=True)
    # Generate a mask for the upper triangle
    mask = np.zeros_like(matrix)
    mask[np.triu_indices_from(mask)] = True
    # Set up the matplotlib figure
    fig, ax = plt.subplots(figsize=(15, 8))
    plt.title('Clinical Features Correlation')
    # Draw the heatmap with the mask and correct aspect ratio
    heatmap = sns.heatmap(matrix, vmax=1.2, square=False, cmap='crest', mask=mask, ax=ax, annot=True, fmt='.2g', linewidths=1)
    return heatmap

def apply_pca(data, target_column, is_training=True, pca_model=None):
    """
    Apply PCA transformation while preserving clinical features and target.
    """
    # Separate target
    target = data[target_column]
    
    # Separate clinical and MRI features
    clinical_features = data.iloc[:, 0:12]
    mri_features = data.iloc[:, 12:]
    
    if is_training:
        # Initialize and fit PCA on the training data
        pca = PCA(n_components=0.95)
        data_pca = pca.fit_transform(mri_features)
        
        # Create DataFrame with meaningful column names for PCA components
        pca_columns = [f'PCA_Component_{i+1}' for i in range(data_pca.shape[1])]
        data_pca = pd.DataFrame(data=data_pca, columns=pca_columns)
        
        # Save column names for test data
        feature_names = list(clinical_features.columns) + list(data_pca.columns)
        
        # Combine all parts
        data_transformed = pd.concat([clinical_features, data_pca, target], axis=1)
        return data_transformed, pca, feature_names
    else:
        # Transform test data using saved PCA model
        data_pca = pca_model.transform(mri_features)
        
        # Create DataFrame with same column names as training
        n_components = pca_model.n_components_
        pca_columns = [f'PCA_Component_{i+1}' for i in range(n_components)]
        data_pca = pd.DataFrame(data=data_pca, columns=pca_columns)
        
        # Combine all parts
        data_transformed = pd.concat([clinical_features, data_pca, target], axis=1)
        return data_transformed

def display_pca_variance(data):
    data, pca = apply_pca(data)
    plt.figure(figsize=(8, 5))
    plt.plot(np.cumsum(pca.explained_variance_ratio_), marker='o')
    plt.title('Cumulative Explained Variance')
    plt.xlabel('Number of Components')
    plt.ylabel('Cumulative Explained Variance')
    plt.grid()
    return plt


### Data splitting

In [9]:
def split_dataset(data, target_column, test_size=0.2, random_state=42):
    """
    Split dataset into training and testing sets.
    
    Parameters:
    -----------
    data : pandas.DataFrame
        The complete dataset including features and target
    target_column : str
        Name of the target column
    test_size : float, default=0.2
        Proportion of the dataset to include in the test split
    random_state : int, default=42
        Random state for reproducibility
    
    Returns:
    --------
    tuple : (X_train, X_test, y_train, y_test)
        Split datasets for training and testing
    """
    # Split features and target
    X = data.drop(target_column, axis=1)
    y = data[target_column]
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=test_size, 
        random_state=random_state,
        stratify=y  # Ensure balanced split for classification
    )
    
    return X_train, X_test, y_train, y_test

### Prepare dataset and target

In [10]:
def prepare_dataset(file_path, target_column, is_training=True):
    """
    Load and prepare the initial dataset.
    """
    # Read data
    data = read_data(file_path)
    
    # Replace 999 with NaN
    data = replace_nan_values(data)
    
    # Drop ID column if it exists
    if 'ID' in data.columns:
        data = drop_columns(data.copy(), ['ID'], 1)
    
    if is_training:
        # Process target variable
        data = prepare_target(data, target_column)
        # Save column names for later use
        os.makedirs('./models/data_preprocessing', exist_ok=True)
        feature_columns = [col for col in data.columns if col != target_column]
        joblib.dump(feature_columns, './models/data_preprocessing/original_columns.joblib')
    else:
        # Load expected columns from training
        try:
            expected_columns = joblib.load('./models/data_preprocessing/original_columns.joblib')
            # Add missing columns with zeros
            for col in expected_columns:
                if col not in data.columns:
                    data[col] = 0
            # Add target column with dummy values for test data
            data[target_column] = 0
            # Reorder columns to match training data
            data = data[expected_columns + [target_column]]
        except FileNotFoundError:
            raise ValueError("Column information not found. Please run training pipeline first.")
    
    # Identify feature types
    feature_data = drop_columns(data.copy(), target_column, 1)
    categorical_features, continuous_features = identify_categorical_continuous_features(feature_data)
    
    print(f"\nDataset Summary:")
    print(f"Shape: {data.shape}")
    
    return data, categorical_features, continuous_features

def prepare_target(data, target_column):
    """
    Prepare target variable by handling missing values and converting to proper type.
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Input dataset
    target_column : str
        Name of the target column
    
    Returns:
    --------
    pandas.DataFrame
        Dataset with processed target variable
    """
    # Convert target to integer type, temporarily using -1 for NaN
    data[target_column] = data[target_column].fillna(-1).astype(int)
    
    # Replace -1 back to NaN
    data.loc[data[target_column] == -1, target_column] = np.nan
    
    return data

### Applying the preprocessing pipeline

In [11]:
class DataPreprocessingPipeline:
    def __init__(self, file_path, is_training, target_column, augmentation_strategy=None):
        """Initialize the pipeline with the data file path."""
        self.file_path = file_path
        self.is_training = is_training
        self.augmentation_strategy = augmentation_strategy
        self.data = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.categorical_features = None
        self.continuous_features = None
        self.target_vars = None
        self.target_column = target_column
        self.pca_model = None
        self.feature_names = None
        
    def load_and_prepare_data(self):
        """Load and prepare the initial dataset."""
        
        # Prepare dataset
        self.data, self.categorical_features, self.continuous_features = prepare_dataset(
            file_path=self.file_path,
            target_column=self.target_column,
            is_training=self.is_training
        )
        
        if self.is_training:
            # Save feature information
            self.feature_names = [col for col in self.data.columns if col != self.target_column]
            os.makedirs('./models/data_preprocessing', exist_ok=True)
            joblib.dump(self.feature_names, './models/data_preprocessing/feature_names.joblib')
        else:
            # Load feature names from training
            self.feature_names = joblib.load('./models/data_preprocessing/feature_names.joblib')
        
        return self
    
    def handle_missing_values(self):
        """Handle missing values using appropriate imputation strategies."""
        self.data = impute_features_and_target(
            self.data,
            self.target_column,
            self.categorical_features,
            self.continuous_features
        )
        return self
    
    def handle_outliers(self, method='clip', z_threshold=3):
        """Handle outliers using the specified method."""
        self.data = handle_z_score_outliers(
            self.data, 
            method=method, 
            z_threshold=z_threshold
        )
        return self
    
    def augment_training_data(self):
        """Apply data augmentation to training data if specified."""
        if not self.is_training or not self.augmentation_strategy:
            return self
            
        print(f"\nApplying {self.augmentation_strategy} augmentation...")
        print(f"Before augmentation - Shape: {self.data.shape}")
        print(f"Class distribution:\n{self.data[self.target_column].value_counts()}")
        
        # Separate features and target
        X = self.data.drop(self.target_column, axis=1)
        y = self.data[self.target_column]
        
        # Apply augmentation
        X_balanced, y_balanced = augment_data(
            X, y, 
            strategy=self.augmentation_strategy
        )
        
        # Combine back into DataFrame
        self.data = pd.concat([
            pd.DataFrame(X_balanced, columns=X.columns),
            pd.Series(y_balanced, name=self.target_column)
        ], axis=1)
        
        print(f"After augmentation - Shape: {self.data.shape}")
        print(f"Class distribution:\n{self.data[self.target_column].value_counts()}")
        
        return self
    
    def normalize_features(self):
        """Apply normalization to the features."""
        self.data = normalise_data(self.data, self.continuous_features)
        return self
    
    def reduce_dimensions(self):
        """Apply PCA to reduce dimensionality of MRI features."""
        os.makedirs('./models/data_preprocessing', exist_ok=True)
        
        if self.is_training:
            # For training data: fit and save PCA model
            self.data, self.pca_model, self.feature_names = apply_pca(data=self.data, target_column=self.target_column, is_training=True)
            
            # Save PCA model and feature names
            joblib.dump(self.pca_model, './models/data_preprocessing/pca_model.joblib')
            joblib.dump(self.feature_names, './models/data_preprocessing/feature_names.joblib')
        else:
            # For test data: load saved PCA model and transform
            try:
                self.pca_model = joblib.load('./models/data_preprocessing/pca_model.joblib')
                self.feature_names = joblib.load('./models/data_preprocessing/feature_names.joblib')
                self.data = apply_pca(data=self.data, target_column=self.target_column, is_training=False, pca_model=self.pca_model)
                
                # Ensure columns match training data
                missing_cols = set(self.feature_names) - set(self.data.columns)
                for col in missing_cols:
                    self.data[col] = 0
                
                # Reorder columns to match training
                self.data = self.data[self.feature_names + ['pCR (outcome)']]
                
            except FileNotFoundError:
                raise ValueError("PCA model not found. Please run training pipeline first.")
        
        return self
    
    def split_data(self, test_size=0.2, random_state=42):
        """Split the data into training and testing sets."""
        if self.is_training:
            self.X_train, self.X_test, self.y_train, self.y_test = split_dataset(
                self.data,
                self.target_column,
                test_size=test_size,
                random_state=random_state
            )
        else:
            # For test data, don't split - just separate features and target
            self.X_test = self.data.drop(self.target_column, axis=1)
            self.y_test = self.data[self.target_column]
            # Set other attributes to None for test data
            self.X_train = None
            self.y_train = None
        return self
    
    def get_processed_data(self):
        """Return the processed and split data."""
        if self.is_training:
            return self.X_train, self.X_test, self.y_train, self.y_test
        else:
            # For test data, return None for training data
            return None, self.X_test, None, self.y_test
    
    def run_pipeline(self):
        """Run the complete preprocessing pipeline."""
        (self
        .load_and_prepare_data()
        .handle_missing_values()
        .handle_outliers()
        .augment_training_data()
        .normalize_features()
        .reduce_dimensions()
        .split_data()
        )
        return self.get_processed_data()

In [12]:
# Initialize and run the pipeline
pipeline = DataPreprocessingPipeline(file_path='../data/TrainDataset2024.xls', is_training=True, target_column='pCR (outcome)', augmentation_strategy='smote')
X_train, X_test, y_train, y_test = pipeline.run_pipeline()

# Create DataFrames for train and test sets with their labels
train_data = pd.DataFrame(X_train)
train_data['pCR (outcome)'] = y_train

test_data = pd.DataFrame(X_test)
test_data['pCR (outcome)'] = y_test

# Concatenate train and test sets
combined_data = pd.concat([train_data, test_data], axis=0)
print("New dataset shape:", combined_data.shape)
display(combined_data)




Dataset Summary:
Shape: (400, 120)

Applying smote augmentation...
Before augmentation - Shape: (400, 120)
Class distribution:
pCR (outcome)
0    316
1     84
Name: count, dtype: int64
After augmentation - Shape: (632, 120)
Class distribution:
pCR (outcome)
1    316
0    316
Name: count, dtype: int64
New dataset shape: (632, 27)


,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,TumourStage,Gene,...,PCA_Component_6,PCA_Component_7,PCA_Component_8,PCA_Component_9,PCA_Component_10,PCA_Component_11,PCA_Component_12,PCA_Component_13,PCA_Component_14,pCR (outcome)
612,0.132801,0.132801,0.132801,0.867199,2.734398,1.867199,1.0,0.000000,1.265602,0.867199,...,-0.255615,2.306287,0.302505,-0.734249,-0.410544,-0.111068,0.312285,-0.807706,-1.778402,1
353,0.000000,0.000000,1.000000,0.000000,3.000000,1.000000,1.0,0.000000,2.000000,0.000000,...,0.758432,0.369647,-0.500542,-0.727722,-0.911664,0.901332,-0.808491,-0.902036,-1.457106,0
0,0.000000,0.000000,0.000000,1.000000,3.000000,3.000000,1.0,1.000000,2.000000,1.000000,...,3.077975,1.328483,-4.604421,0.566355,-1.433317,-2.189594,1.413369,0.157920,-3.178941,1
97,0.000000,0.000000,0.000000,1.000000,2.000000,1.000000,1.0,1.000000,2.000000,1.000000,...,1.379061,0.275397,1.327597,0.115218,0.247911,0.802660,0.118586,-0.693596,0.367737,0
613,0.636404,0.636404,0.636404,0.363596,1.727193,1.363596,1.0,0.363596,2.272807,0.363596,...,-0.416776,-0.303488,-0.289124,-0.325761,0.132904,0.794607,-0.978691,0.169059,-0.477471,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,0.005185,0.005185,0.005185,0.994815,3.000000,2.989630,1.0,0.000000,3.994815,0.994815,...,0.049580,-1.156276,0.642646,-0.736834,-1.140305,-0.034129,0.641446,-0.408973,-0.431840,1
40,1.000000,1.000000,0.000000,0.000000,2.000000,1.000000,1.0,1.000000,2.000000,0.000000,...,-1.212231,-0.258955,-0.841753,-0.000465,0.569491,-1.351783,0.704874,-1.582993,-0.367727,0
485,0.000000,0.000000,0.434394,0.565606,3.000000,2.434394,1.0,0.434394,1.434394,0.565606,...,0.859844,1.304509,0.108302,-0.403253,-0.383612,-0.348634,-0.010192,-0.848349,-1.369505,1
25,1.000000,0.000000,1.000000,0.000000,2.000000,1.000000,1.0,1.000000,2.000000,1.000000,...,0.305830,-1.902182,-0.419410,0.038995,0.423305,-0.817338,0.689675,-0.514102,-0.307187,0


### Split features and target

In [13]:
# Split features and target
X = combined_data.drop('pCR (outcome)', axis=1)
y = combined_data['pCR (outcome)']

### K-Fold Cross Validation

In [14]:
def get_base_models_and_params(random_state=42):
    """Get base models and their parameter grids."""
    return {
        'Logistic Regression': (
            LogisticRegression(random_state=random_state, class_weight='balanced'),
            {
                'C': [0.001, 0.01, 0.1, 1, 10, 100],
                'solver': ['lbfgs', 'liblinear'],
                'max_iter': [2000]
            }
        ),
        'MLP': (
            MLPClassifier(random_state=random_state, early_stopping=True),
            {
                'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
                'activation': ['relu', 'tanh'],
                'alpha': [0.0001, 0.001, 0.01],
                'learning_rate': ['constant', 'adaptive']
            }
        ),
        'SVM': (
            SVC(random_state=random_state, class_weight='balanced'),
            {
                'C': [0.1, 1, 10],
                'kernel': ['linear', 'rbf'],
                'gamma': ['scale', 'auto', 0.1, 1]
            }
        ),
        'Decision Tree': (
            DecisionTreeClassifier(random_state=random_state, class_weight='balanced'),
            {
                'max_depth': [3, 5, 7, None],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 4]
            }
        )
    }

def evaluate_model(model, X_train, X_test, y_train, y_test, cv):
    """Evaluate a single model using cross-validation and test set."""
    cv_scores = []
    
    # Cross-validation
    for train_idx, val_idx in cv.split(X_train, y_train):
        model.fit(X_train[train_idx], y_train[train_idx])
        y_pred = model.predict(X_train[val_idx])
        scores = {
            'accuracy': accuracy_score(y_train[val_idx], y_pred),
            'precision': precision_score(y_train[val_idx], y_pred),
            'recall': recall_score(y_train[val_idx], y_pred),
            'f1': f1_score(y_train[val_idx], y_pred)
        }
        cv_scores.append(scores)
    
    # Test set evaluation
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    test_scores = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred)
    }
    
    return {'cv_scores': cv_scores, 'test_scores': test_scores}

def run_evaluation(X, y, optimize=True):
    """Run complete evaluation pipeline with optional optimization."""
    # Prepare data
    X, y = np.asarray(X), np.asarray(y)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )
    
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    results = {}
    
    # Get models and their parameter grids
    models_and_params = get_base_models_and_params()
    
    # Evaluate each model
    for name, (model, param_grid) in models_and_params.items():
        if optimize:
            print(f"\nOptimizing {name}...")
            grid_search = GridSearchCV(
                model, param_grid, cv=5, scoring='f1', n_jobs=-1
            )
            grid_search.fit(X_train, y_train)
            model = grid_search.best_estimator_
            print(f"Best parameters: {grid_search.best_params_}")
            print(f"Best CV score: {grid_search.best_score_:.3f}")
        
        results[name] = evaluate_model(
            model, X_train, X_test, y_train, y_test, cv
        )
    
    # Create results DataFrames
    cv_data = [{
        'Model': name,
        **{metric: f"{np.mean([fold[metric] for fold in scores['cv_scores']]):.3f} ± "
                  f"{np.std([fold[metric] for fold in scores['cv_scores']]):.3f}"
           for metric in ['accuracy', 'precision', 'recall', 'f1']}
    } for name, scores in results.items()]
    
    test_data = [{
        'Model': name,
        **{metric: f"{scores['test_scores'][metric]:.3f}"
           for metric in ['accuracy', 'precision', 'recall', 'f1']}
    } for name, scores in results.items()]
    
    # Display results
    cv_df = pd.DataFrame(cv_data).set_index('Model')
    test_df = pd.DataFrame(test_data).set_index('Model')
    
    print("\nCross-validation Results:")
    display(cv_df.style.set_properties(**{'text-align': 'center'})
           .set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}]))
    
    print("\nTest Set Results:")
    display(test_df.style.set_properties(**{'text-align': 'center'})
           .set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}]))
    
    return results, cv_df, test_df

In [15]:
# Run evaluation
results, cv_df, test_df = run_evaluation(X, y, optimize=True)


Optimizing Logistic Regression...
Best parameters: {'C': 1, 'max_iter': 2000, 'solver': 'lbfgs'}
Best CV score: 0.735

Optimizing MLP...
Best parameters: {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (100, 50), 'learning_rate': 'constant'}
Best CV score: 0.758

Optimizing SVM...
Best parameters: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
Best CV score: 0.883

Optimizing Decision Tree...
Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10}
Best CV score: 0.763

Cross-validation Results:


,accuracy,precision,recall,f1
Model,,,,
Logistic Regression,0.746 ± 0.046,0.735 ± 0.048,0.775 ± 0.077,0.753 ± 0.049
MLP,0.759 ± 0.040,0.743 ± 0.064,0.819 ± 0.122,0.769 ± 0.049
SVM,0.881 ± 0.045,0.863 ± 0.060,0.913 ± 0.030,0.886 ± 0.039
Decision Tree,0.766 ± 0.064,0.787 ± 0.081,0.739 ± 0.091,0.759 ± 0.070



Test Set Results:


,accuracy,precision,recall,f1
Model,,,,
Logistic Regression,0.693,0.654,0.810,0.723
MLP,0.780,0.746,0.841,0.791
SVM,0.858,0.800,0.952,0.870
Decision Tree,0.787,0.765,0.825,0.794


### Testing preprocessing on test dataset

In [16]:
# Initialize and run the pipeline
pipeline = DataPreprocessingPipeline(file_path='../data/TestDatasetExample.xls', is_training=False, target_column='pCR (outcome)')
X_train, X_test, y_train, y_test = pipeline.run_pipeline()

# Create DataFrames for train and test sets with their labels
train_data = pd.DataFrame(X_train)
train_data['pCR (outcome)'] = y_train

test_data = pd.DataFrame(X_test)
test_data['pCR (outcome)'] = y_test

# Concatenate train and test sets
combined_data = pd.concat([train_data, test_data], axis=0)
print("New dataset shape:", combined_data.shape)
display(combined_data)




Dataset Summary:
Shape: (3, 120)
New dataset shape: (3, 27)


,pCR (outcome),ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,TumourStage,...,PCA_Component_5,PCA_Component_6,PCA_Component_7,PCA_Component_8,PCA_Component_9,PCA_Component_10,PCA_Component_11,PCA_Component_12,PCA_Component_13,PCA_Component_14
0,0,0.0,0.0,0.0,1.0,3.0,3.0,1.0,0.0,2.0,...,-4.342735e+06,9.228345e+06,-1.293895e+06,5.815474e+06,-6.093675e+06,-2.006505e+06,-1.153095e+06,837867.443460,-997979.710300,2.592124e+06
1,0,0.0,0.0,1.0,0.0,2.0,1.0,1.0,0.0,3.0,...,-1.787514e+05,3.786511e+05,-5.293761e+04,2.370056e+05,-2.462757e+05,-7.973069e+04,-4.448908e+04,30189.121622,-39654.471274,1.024651e+05
2,0,0.0,0.0,0.0,1.0,3.0,3.0,1.0,1.0,4.0,...,-3.456918e+06,7.304004e+06,-1.025277e+06,4.610810e+06,-4.812573e+06,-1.588671e+06,-8.905133e+05,660230.151524,-791301.738564,2.054329e+06


In [17]:
# Check if the preprocessing of both training and test dataset have the same name features
train_features = train_data.columns.tolist()
test_features = test_data.columns.tolist()

# Find the features that are not common to both datasets
missing_train_features = [feature for feature in test_features if feature not in train_features]
missing_test_features = [feature for feature in train_features if feature not in test_features]

# Print the results
if missing_train_features or missing_test_features:
    print("Features not common to both datasets:")
    if missing_train_features:
        print("Missing in training dataset:", missing_train_features)
    if missing_test_features:
        print("Missing in test dataset:", missing_test_features)
else:
    print("Both datasets have the same features.")

Features not common to both datasets:
Missing in training dataset: ['ER', 'PgR', 'HER2', 'TrippleNegative', 'ChemoGrade', 'Proliferation', 'HistologyType', 'LNStatus', 'TumourStage', 'Gene', 'RelapseFreeSurvival (outcome)', 'Age', 'PCA_Component_1', 'PCA_Component_2', 'PCA_Component_3', 'PCA_Component_4', 'PCA_Component_5', 'PCA_Component_6', 'PCA_Component_7', 'PCA_Component_8', 'PCA_Component_9', 'PCA_Component_10', 'PCA_Component_11', 'PCA_Component_12', 'PCA_Component_13', 'PCA_Component_14']
